In [1]:
import pandas as pd
import numpy as np 
import os
import warnings
warnings.filterwarnings('ignore')
from scipy import stats
from sklearn.model_selection import cross_val_score, train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder, RobustScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, f1_score, make_scorer, roc_auc_score, mean_squared_error,mean_absolute_error
from sklearn.cluster import KMeans
from scipy import stats
from sklearn.model_selection import GridSearchCV
from prophet.diagnostics import cross_validation
import joblib
# import optuna
# import xgboost as xgb
import catboost
import lightgbm as lgb
from skopt import BayesSearchCV
from prophet import Prophet
from prophet.diagnostics import performance_metrics
import xgboost as xgb

In [2]:
def y_changer(data):
    data = data.rename(columns = {"y": "평균기온", "월":"months"})
    return data

In [3]:
data = pd.read_csv('./prep_data.csv')
data.ds = pd.to_datetime(data.ds)
data = data.rename(columns = {"평균기온":"y"})
data['days'] = data.ds.dt.day
data['years'] = data.ds.dt.year
summit_form = pd.read_csv('./sample_submission.csv')
regressors = [x for x in data.columns if x != "y" and x != "ds"]
data = y_changer(data)

In [4]:
data.columns

Index(['최고기온', '최저기온', '일교차', '강수량', '평균습도', '평균풍속', '일조합', '일사합', '일조율',
       '평균기온', 'months', 'ds', 'days', 'years'],
      dtype='object')

In [5]:
X = data.drop(["평균기온",'일교차', '일조합','ds'], axis = 1)
y = data.평균기온

In [33]:
y_new = y[365:]

In [34]:
X_new = X[:-365]

In [12]:
new

22646

In [13]:
len(y_new)

22646

In [40]:
X_train, X_test = X_new[:-365], X_new[-365:]
y_train, y_test = y_new[:-365], y_new[-365:]

In [41]:
X_test

,최고기온,최저기온,강수량,평균습도,평균풍속,일사합,일조율,months,days,years
22281,1.6,-9.8,0.61,64.0,2.0,9.54,67.7,1.0,1,2021
22282,-1.4,-8.4,0.18,38.5,2.6,11.34,93.8,1.0,2,2021
22283,-2.0,-9.1,0.09,45.0,2.0,8.80,56.7,1.0,3,2021
22284,0.3,-8.4,0.00,51.4,1.7,7.44,47.4,1.0,4,2021
22285,-2.1,-9.9,0.00,52.8,2.9,10.46,88.7,1.0,5,2021
...,...,...,...,...,...,...,...,...,...,...
22641,-3.9,-12.9,0.00,60.9,1.7,6.09,39.6,12.0,27,2021
22642,-0.9,-8.5,0.66,73.8,2.2,4.32,17.7,12.0,28,2021
22643,5.9,-3.8,0.20,72.9,2.6,4.64,18.8,12.0,29,2021
22644,0.2,-6.8,0.00,48.5,3.3,9.14,76.0,12.0,30,2021


In [45]:
import xgboost as xgb
from sklearn.model_selection import cross_val_score
from skopt import BayesSearchCV



# 하이퍼파라미터 탐색 범위 정의
param_space = {
    'learning_rate': (0.01, 1.00, 'uniform'),
    'max_depth': (3, 12),
    'n_estimators': (100, 1000),
    'subsample': (0.10, 1.00, 'uniform'),
    'colsample_bytree': (0.10, 1.00, 'uniform'),
    'gamma': (0, 1.0, 'uniform'),
}

# XGBoost 모델 초기화
model = xgb.XGBRegressor(objective="reg:squarederror", random_state=42)

# BayesSearchCV를 사용하여 베이지안 최적화 수행
opt = BayesSearchCV(
    model,
    param_space,
    scoring='neg_mean_absolute_error',
    n_iter=50,  # 반복 횟수
    cv=5,  # 교차 검증 폴드 수
    n_jobs=1,  # 병렬 처리를 위한 CPU 코어 수 (-1은 모든 가능한 코어 사용)
)

# 최적의 하이퍼파라미터로 모델 학습
opt.fit(X_train, y_train)

# 최적의 하이퍼파라미터 출력
print("Best Parameters:", opt.best_params_)

# 최적의 모델 평가 (여기서는 교차 검증 점수 출력)
print("Cross-Validation Score:", -opt.best_score_)


Best Parameters: OrderedDict([('colsample_bytree', 0.8780741118210806), ('gamma', 0), ('learning_rate', 0.12473347174963892), ('max_depth', 8), ('n_estimators', 456), ('subsample', 0.6080385860401739)])
Cross-Validation Score: 2.758964698078302


In [47]:
opt.best_estimators

AttributeError: 'BayesSearchCV' object has no attribute 'best_estimators'

In [48]:
best_model = opt.best_estimator_

In [58]:
pred = best_model.predict(X_test)

mean_absolute_error(y_test, pred)

mae = mean_absolute_error(y_test, pred)

In [118]:
def select_label(data, labels, label):

    # X = data.drop(["평균기온",'일교차', '일조합','ds', label], axis = 1)
    X = data[labels].drop([label], axis = 1)
    y = data[label]
    
    return X, y 

def train_test_data(X,y):
    
    X_train, X_test = X[:-365], X[-365:]
    y_train, y_test = y[:-365], y[-365:]
    
    return X_train, X_test, y_train, y_test

def make_model(X_train, y_train):
    
    # 하이퍼파라미터 탐색 범위 정의
    param_space = {
        'learning_rate': (0.01, 1.00, 'uniform'),
        'max_depth': (3, 12),
        'n_estimators': (100, 1000),
        'subsample': (0.10, 1.00, 'uniform'),
        'colsample_bytree': (0.10, 1.00, 'uniform'),
        'gamma': (0, 1.0, 'uniform'),
    }

    # XGBoost 모델 초기화
    model = xgb.XGBRegressor(objective="reg:squarederror", n_jobs = -1, random_state=42)

    # BayesSearchCV를 사용하여 베이지안 최적화 수행
    opt = BayesSearchCV(
        model,
        param_space,
        scoring='neg_mean_absolute_error',
        n_iter=50,  # 반복 횟수
        cv=5,  # 교차 검증 폴드 수
        # n_jobs=1,  # 병렬 처리를 위한 CPU 코어 수 (-1은 모든 가능한 코어 사용)
    )

    # 최적의 하이퍼파라미터로 모델 학습
    opt.fit(X_train, y_train)

    # 최적의 하이퍼파라미터 출력
    print("Best Parameters:", opt.best_params_)

    # 최적의 모델 평가 (여기서는 교차 검증 점수 출력)
    print("Cross-Validation Score:", -opt.best_score_)
    
    return opt
    

def socoring(opt, y_test, X_test):
    
    pred = opt.best_estimator_.predict(X_test)

    mean_absolute_error(y_test, pred)

    mae = mean_absolute_error(y_test, pred)
    print(mae)
    
    return mae

def progress(data, labels):
    
    
    data = data
    
    labels = labels
    
    for label in labels:
    
        X, y = select_label(data, labels, label)

        X_train,X_test, y_train, y_test = train_test_data(X,y)

        opt = make_model(X_train,y_train)

        label = label + "_model"
        
        print(label)

        globals()[label] = opt

        mae = socoring(opt, y_test, X_test)

        return print(mae)

def modeling(params, X_train, y_train):
    
    model = xgb.XGBRegressor(objective="reg:squarederror", **params, n_jobs = -1, random_state=42)
    
    model.fit(X_train, y_train)
    
    return model

def model_predict(data, label, params):
    
    data = data
    
    label = label
    
    X, y = select_label(data, label)
    
    X_train,X_test, y_train, y_test = train_test_data(X,y)
    
    model = modeling(params, X_train, y_train)
    
    model_pred = model.predict(X_test)
    
    return model_pred
    

In [113]:
avg_tmp = dict([('colsample_bytree', 0.7489488665614384), ('gamma', 0), ('learning_rate', 0.2303112080128232), ('max_depth', 3), ('n_estimators', 1000), ('subsample', 1.0)])
high_tmp = dict([('colsample_bytree', 0.785843038567719), ('gamma', 1), ('learning_rate', 0.01), ('max_depth', 3), ('n_estimators', 1000), ('subsample', 0.36837214707260824)])
low_tmp = dict([('colsample_bytree', 0.9307064570326005), ('gamma', 0), ('learning_rate', 0.01), ('max_depth', 5), ('n_estimators', 1000), ('subsample', 1.0)])
avg_hum = dict([('colsample_bytree', 1.0), ('gamma', 0), ('learning_rate', 0.011022102502535366), ('max_depth', 4), ('n_estimators', 1000), ('subsample', 0.11499728444446672)])

In [114]:
avg_tmp

{'colsample_bytree': 0.7489488665614384,
 'gamma': 0,
 'learning_rate': 0.2303112080128232,
 'max_depth': 3,
 'n_estimators': 1000,
 'subsample': 1.0}

In [102]:
labels = ["평균기온", "최고기온", "최저기온", "평균습도"]
for label in labels :
    progress(data, labels)

Best Parameters: OrderedDict([('colsample_bytree', 0.7489488665614384), ('gamma', 0), ('learning_rate', 0.2303112080128232), ('max_depth', 3), ('n_estimators', 1000), ('subsample', 1.0)])
Cross-Validation Score: 0.39732766430568645
0.3690247294970164
0.3690247294970164
Best Parameters: OrderedDict([('colsample_bytree', 0.785843038567719), ('gamma', 1), ('learning_rate', 0.01), ('max_depth', 3), ('n_estimators', 1000), ('subsample', 0.36837214707260824)])
Cross-Validation Score: 0.39090319278662566
0.3763858020795535
0.3763858020795535
Best Parameters: OrderedDict([('colsample_bytree', 0.9307064570326005), ('gamma', 0), ('learning_rate', 0.01), ('max_depth', 5), ('n_estimators', 1000), ('subsample', 1.0)])
Cross-Validation Score: 0.3902358750681939
0.37210015173235994
0.37210015173235994
Best Parameters: OrderedDict([('colsample_bytree', 1.0), ('gamma', 0), ('learning_rate', 0.011022102502535366), ('max_depth', 4), ('n_estimators', 1000), ('subsample', 0.11499728444446672)])
Cross-Valid

In [104]:
print(최고기온_model.best_params_)
print(최저기온_model.best_params_)
print(평균습도_model.best_params_)
print(평균기온_model.best_params_)

OrderedDict([('colsample_bytree', 1.0), ('gamma', 0), ('learning_rate', 0.01), ('max_depth', 3), ('n_estimators', 1000), ('subsample', 1.0)])
OrderedDict([('colsample_bytree', 1.0), ('gamma', 1), ('learning_rate', 0.219519027392961), ('max_depth', 8), ('n_estimators', 1000), ('subsample', 0.5209965334687984)])
OrderedDict([('colsample_bytree', 1.0), ('gamma', 0), ('learning_rate', 0.01), ('max_depth', 5), ('n_estimators', 978), ('subsample', 0.6873473201351862)])
OrderedDict([('colsample_bytree', 1.0), ('gamma', 0), ('learning_rate', 0.011022102502535366), ('max_depth', 4), ('n_estimators', 1000), ('subsample', 0.11499728444446672)])


In [125]:
label_dic = {"최고기온" : high_tmp, "최저기온" : low_tmp, "평균습도" : avg_hum}

In [126]:

X, y = select_label(data, labels, "평균기온")

X_train,X_test, y_train, y_test = train_test_data(X,y)

base_m = modeling(avg_tmp, X_train, y_train)\

for k,v in label_dic.items() :
    
    X1, y1 = select_label(data, labels, k)
    
    X_train1,X_test1, y_train1, y_test1 = train_test_data(X1,y1)
    
    print(v)
    
    m = modeling(v, X_train1, y_train1)
    
    X_test[k] = m.predict(X_test1)
    

# y_pred = base_m.predict(X_test)

# mae = mean_absolute_error(y_test, pred)

# print(mae)

{'colsample_bytree': 0.785843038567719, 'gamma': 1, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 1000, 'subsample': 0.36837214707260824}
{'colsample_bytree': 0.9307064570326005, 'gamma': 0, 'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 1000, 'subsample': 1.0}
{'colsample_bytree': 1.0, 'gamma': 0, 'learning_rate': 0.011022102502535366, 'max_depth': 4, 'n_estimators': 1000, 'subsample': 0.11499728444446672}


In [127]:
y_pred = base_m.predict(X_test)

mae = mean_absolute_error(y_test, pred)

print(mae)

2.8153716568097678


In [128]:
X_test

,최고기온,최저기온,평균습도
22646,-0.155919,-8.238092,60.601475
22647,3.271398,-5.182496,56.391415
22648,2.114505,-5.983513,57.746731
22649,2.158552,-6.311744,54.248131
22650,1.312312,-6.560971,55.886009
...,...,...,...
23006,1.562991,-6.547873,54.769093
23007,1.144488,-6.849960,56.458641
23008,1.172052,-6.911513,56.207546
23009,2.602852,-5.330601,57.111187


In [ ]:

X, y = select_label(data, labels, "평균기온")

# X_train,X_test, y_train, y_test = train_test_data(X,y)
X_test = pd.DataFrame()

base_m = modeling(avg_tmp, X, y)

for k,v in label_dic.items() :
    
    X1, y1 = select_label(data, labels, k)
    
    # X_train1,X_test1, y_train1, y_test1 = train_test_data(X1,y1)
    
    print(v)
    
    m = modeling(v, X1, y1)
    
    X_test[k] = m.predict(X_test1)
    

# y_pred = base_m.predict(X_test)

# mae = mean_absolute_error(y_test, pred)

# print(mae)

In [22]:
def base_select_label(data, labels, label):
    
    X = data[["years", "months", "days"]]
    y = data[label]
    
    return X, y 

def train_test_data(X,y, level):
    
    X_train, X_test = X[:-level], X[-level:]
    y_train, y_test = y[:-level], y[-level:]
    
    return X_train, X_test, y_train, y_test

def progress(data, labels):
    
    
    data = data
    
    labels = labels
    
    for label in labels:
    
        X, y = select_label(data, labels, label)

        X_train,X_test, y_train, y_test = train_test_data(X,y)

        opt = make_model(X_train,y_train)

        label = label + "_model"
        
        print(label)

        globals()[label] = opt
    
def make_model(X_train, y_train, X_test, y_test):
    
    # 하이퍼파라미터 탐색 범위 정의
    param_space = {
        'learning_rate': (0.0001, 0.1, 'uniform'),
        'max_depth': (3, 12),
        'n_estimators': (100, 3000),
        'subsample': (0.10, 1.00, 'uniform'),
        'colsample_bytree': (0.10, 1.00, 'uniform'),
        'gamma': (0, 10.0, 'uniform'),
        'eval_set' : [(X_test, y_test)]
    }

    # XGBoost 모델 초기화
    model = xgb.XGBRegressor(objective="reg:squarederror", eval_metric='mae', n_jobs = -1, random_state=42)

    # BayesSearchCV를 사용하여 베이지안 최적화 수행
    opt = BayesSearchCV(
        model,
        param_space,
        scoring='neg_mean_absolute_error',
        n_iter=50,  # 반복 횟수
        cv=5,  # 교차 검증 폴드 수
        n_jobs=1,  # 병렬 처리를 위한 CPU 코어 수 (-1은 모든 가능한 코어 사용)
    )
    
    # 최적의 하이퍼파라미터로 모델 학습
    opt.fit(X_train, y_train)

    # 최적의 하이퍼파라미터 출력|
    print("Best Parameters:", opt.best_params_)

    # 최적의 모델 평가 (여기서는 교차 검증 점수 출력)
    print("Cross-Validation Score:", -opt.best_score_)
    
    return opt
    

In [23]:
data.columns

Index(['최고기온', '최저기온', '일교차', '강수량', '평균습도', '평균풍속', '일조합', '일사합', '일조율',
       '평균기온', 'months', 'ds', 'days', 'years'],
      dtype='object')

In [24]:
labels = ["최고기온", "최저기온", "평균습도", "평균풍속","일사합", "일조율", "강수량"]

In [25]:
X_test

,years,months,days
21916,2020,1,2
21917,2020,1,3
21918,2020,1,4
21919,2020,1,5
21920,2020,1,6
...,...,...,...
23006,2022,12,27
23007,2022,12,28
23008,2022,12,29
23009,2022,12,30


In [26]:
X['months'] = X['months'].astype(int)

In [32]:
import optuna

X_train,X_test, y_train, y_test = train_test_data(X,y, 1095)

In [37]:
def make_model_opt(X_train, y_train, X_test, y_test):
    
    X_train = X_train
    y_train = y_train
    X_test = X_test
    y_test = y_test
    
    

    def objective(trial):
        params = {
            'objective': 'reg:squarederror',
            'eval_metric': 'mae',
            'booster': trial.suggest_categorical('booster', ['gbtree', 'gblinear', 'dart']),
            'lambda': trial.suggest_loguniform('lambda', 1e-8, 1.0),
            'alpha': trial.suggest_loguniform('alpha', 1e-8, 1.0),
            'max_depth': trial.suggest_int('max_depth', 3, 10),
            'subsample': trial.suggest_uniform('subsample', 0.1, 1.0),
            'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.1, 1.0),
            'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.1),
            'n_estimators': trial.suggest_int('n_estimators', 50, 300)
        }

        model = xgb.XGBRegressor(**params)
        model.fit(X_train, y_train, eval_set=[(X_test, y_test)], early_stopping_rounds=10, verbose=False)
        predictions = model.predict(X_test)
        mae = mean_absolute_error(y_test, predictions)
        return mae



    # Optuna 베이지안 최적화를 사용하여 파라미터 튜닝 실행
    study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=42))
    study.optimize(objective, n_trials=100)

    # 최적의 파라미터 출력
    print('Best trial:')
    print('  Value: {}'.format(study.best_trial.value))
    print('  Params: ')
    for key, value in study.best_trial.params.items():
        print('    {}: {}'.format(key, value))
        
    return study

In [38]:
for label in labels:

    X, y = base_select_label(data, labels, label)
    
    X['months'] = X['months'].astype(int)

    X_train,X_test, y_train, y_test = train_test_data(X,y, 1095)

    study = make_model_opt(X_train,y_train, X_test, y_test)
    
    globals()[label+"_model"] = study

[I 2023-12-26 15:57:23,182] A new study created in memory with name: no-name-b300644d-95a8-4278-9f63-0c90383d4a93


[15:57:23] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "colsample_bytree", "max_depth", "subsample" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




[I 2023-12-26 15:57:23,434] Trial 0 finished with value: 8.578939764815377 and parameters: {'booster': 'gblinear', 'lambda': 0.0006155564318973012, 'alpha': 1.77071686435378e-07, 'max_depth': 4, 'subsample': 0.15227525095137953, 'colsample_bytree': 0.8795585311974417, 'learning_rate': 0.015930522616241012, 'n_estimators': 227}. Best is trial 0 with value: 8.578939764815377.
[I 2023-12-26 15:57:23,573] Trial 1 finished with value: 8.742557395691197 and parameters: {'booster': 'gblinear', 'lambda': 4.997040685255803e-07, 'alpha': 2.8483918709107956e-07, 'max_depth': 4, 'subsample': 0.373818018663584, 'colsample_bytree': 0.5722807884690141, 'learning_rate': 0.007309539835912915, 'n_estimators': 123}. Best is trial 0 with value: 8.578939764815377.


[15:57:23] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "colsample_bytree", "max_depth", "subsample" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




[I 2023-12-26 15:57:23,778] Trial 2 finished with value: 9.972721559271966 and parameters: {'booster': 'gbtree', 'lambda': 8.528933855762793e-06, 'alpha': 4.452048365748842e-05, 'max_depth': 9, 'subsample': 0.2797064039425238, 'colsample_bytree': 0.5628109945722505, 'learning_rate': 0.015304852121831466, 'n_estimators': 61}. Best is trial 0 with value: 8.578939764815377.
[I 2023-12-26 15:57:24,293] Trial 3 finished with value: 4.736537847823747 and parameters: {'booster': 'gbtree', 'lambda': 0.39001768308022033, 'alpha': 0.530953226900921, 'max_depth': 9, 'subsample': 0.3741523922560336, 'colsample_bytree': 0.1879049026057455, 'learning_rate': 0.0233596350262616, 'n_estimators': 160}. Best is trial 3 with value: 4.736537847823747.
[I 2023-12-26 15:57:24,413] Trial 4 finished with value: 8.661733107719247 and parameters: {'booster': 'gblinear', 'lambda': 0.1881755597772026, 'alpha': 1.1755466083160747e-06, 'max_depth': 8, 'subsample': 0.3805399684804699, 'colsample_bytree': 0.5680612190

...

[16:14:27] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "colsample_bytree", "max_depth", "subsample" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




[I 2023-12-26 16:14:27,600] Trial 68 finished with value: 4.6929247393455675 and parameters: {'booster': 'dart', 'lambda': 1.5119179833684767e-07, 'alpha': 0.17771178449483854, 'max_depth': 9, 'subsample': 0.9303177328318222, 'colsample_bytree': 0.8800376762401113, 'learning_rate': 0.02205335036234798, 'n_estimators': 55}. Best is trial 27 with value: 4.681405650574323.
[I 2023-12-26 16:14:28,056] Trial 69 finished with value: 4.690762731480271 and parameters: {'booster': 'dart', 'lambda': 1.206370051943443e-05, 'alpha': 0.023225595157700705, 'max_depth': 5, 'subsample': 0.9993703439780789, 'colsample_bytree': 0.927199011788822, 'learning_rate': 0.0074981579295926104, 'n_estimators': 216}. Best is trial 27 with value: 4.681405650574323.
[I 2023-12-26 16:14:28,223] Trial 70 finished with value: 4.712398601090527 and parameters: {'booster': 'gbtree', 'lambda': 0.0019058337751269123, 'alpha': 0.0012514464444204206, 'max_depth': 7, 'subsample': 0.8095504597934782, 'colsample_bytree': 0.657

[16:14:29] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "colsample_bytree", "max_depth", "subsample" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




[I 2023-12-26 16:14:30,421] Trial 81 finished with value: 4.6897197584430925 and parameters: {'booster': 'dart', 'lambda': 0.00039450544353637567, 'alpha': 0.00453729859467473, 'max_depth': 7, 'subsample': 0.9384241363895744, 'colsample_bytree': 0.742053887747877, 'learning_rate': 0.015041979286086313, 'n_estimators': 222}. Best is trial 27 with value: 4.681405650574323.
[I 2023-12-26 16:14:30,561] Trial 82 finished with value: 4.686230224676872 and parameters: {'booster': 'dart', 'lambda': 0.0001275247805995024, 'alpha': 6.442168043717022e-05, 'max_depth': 7, 'subsample': 0.9084681521461481, 'colsample_bytree': 0.8391852204223347, 'learning_rate': 0.07264678808152095, 'n_estimators': 245}. Best is trial 27 with value: 4.681405650574323.
[I 2023-12-26 16:14:30,698] Trial 83 finished with value: 4.687017273643789 and parameters: {'booster': 'dart', 'lambda': 0.00010101371368620661, 'alpha': 6.795446195275846e-05, 'max_depth': 7, 'subsample': 0.9099385505080765, 'colsample_bytree': 0.787

[16:14:32] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "colsample_bytree", "max_depth", "subsample" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


Best trial:
  Value: 4.681405650574323
  Params: 
    booster: dart
    lambda: 7.68022388812152e-05
    alpha: 0.006944741395221721
    max_depth: 8
    subsample: 0.9525844786717498
    colsample_bytree: 0.7463370417524927
    learning_rate: 0.08904839593197302
    n_estimators: 215


In [28]:
for label in labels:

    X, y = base_select_label(data, labels, label)
    
    X['months'] = X['months'].astype(int)

    X_train,X_test, y_train, y_test = train_test_data(X,y, 1095)

    opt = make_model(X_train,y_train, X_test, y_test)
    
    globals()[label+"_model"] = opt

TypeError: unhashable type: 'DataFrame'

In [ ]:
make_model(X_train, y_train)

In [42]:
print('  Value: {}'.format(최고기온_model.best_trial.value))

  Value: 3.081674730723307


In [ ]:
365*3

1095

In [39]:
# X = data[["years", "months", "days","최고기온", "최저기온", "평균습도", "평균풍속","일사합", "일조율", "강수량"]]
# X = data[["years", "months", "days","최고기온", "최저기온", "평균습도", "평균풍속","일사합","강수량"]]
# X = data[["years", "months", "days","최고기온", "최저기온", "평균습도", "평균풍속", "일조율", "강수량"]]
# X = data[["years", "months", "days","최고기온", "최저기온", "평균습도","일사합","강수량"]]
X = data[["years", "months", "days","최고기온", "최저기온", "평균습도", "평균풍속","일조율", "강수량"]]
y = data["평균기온"]

X_train,X_test, y_train, y_test = train_test_data(X,y, 1095)

In [43]:
main_model = make_model_opt(X_train, y_train, X_test, y_test)

[I 2023-12-26 16:24:28,915] A new study created in memory with name: no-name-ec7c2258-800e-4aa8-bc22-ecea31795058


[16:24:28] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "colsample_bytree", "max_depth", "subsample" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




[I 2023-12-26 16:24:29,207] Trial 0 finished with value: 0.8503493992909449 and parameters: {'booster': 'gblinear', 'lambda': 0.0006155564318973012, 'alpha': 1.77071686435378e-07, 'max_depth': 4, 'subsample': 0.15227525095137953, 'colsample_bytree': 0.8795585311974417, 'learning_rate': 0.015930522616241012, 'n_estimators': 227}. Best is trial 0 with value: 0.8503493992909449.
[I 2023-12-26 16:24:29,372] Trial 1 finished with value: 3.5696285768751412 and parameters: {'booster': 'gblinear', 'lambda': 4.997040685255803e-07, 'alpha': 2.8483918709107956e-07, 'max_depth': 4, 'subsample': 0.373818018663584, 'colsample_bytree': 0.5722807884690141, 'learning_rate': 0.007309539835912915, 'n_estimators': 123}. Best is trial 0 with value: 0.8503493992909449.


[16:24:29] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "colsample_bytree", "max_depth", "subsample" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




[I 2023-12-26 16:24:29,775] Trial 2 finished with value: 5.6718128581161364 and parameters: {'booster': 'gbtree', 'lambda': 8.528933855762793e-06, 'alpha': 4.452048365748842e-05, 'max_depth': 9, 'subsample': 0.2797064039425238, 'colsample_bytree': 0.5628109945722505, 'learning_rate': 0.015304852121831466, 'n_estimators': 61}. Best is trial 0 with value: 0.8503493992909449.
[I 2023-12-26 16:24:30,390] Trial 3 finished with value: 2.589454817861557 and parameters: {'booster': 'gbtree', 'lambda': 0.39001768308022033, 'alpha': 0.530953226900921, 'max_depth': 9, 'subsample': 0.3741523922560336, 'colsample_bytree': 0.1879049026057455, 'learning_rate': 0.0233596350262616, 'n_estimators': 160}. Best is trial 0 with value: 0.8503493992909449.
[I 2023-12-26 16:24:30,531] Trial 4 finished with value: 2.8972473121260944 and parameters: {'booster': 'gblinear', 'lambda': 0.1881755597772026, 'alpha': 1.1755466083160747e-06, 'max_depth': 8, 'subsample': 0.3805399684804699, 'colsample_bytree': 0.568061

...

[16:32:01] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "colsample_bytree", "max_depth", "subsample" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




[I 2023-12-26 16:32:01,895] Trial 77 finished with value: 0.982001479791656 and parameters: {'booster': 'gblinear', 'lambda': 2.275057696880392e-06, 'alpha': 0.09143819394747513, 'max_depth': 6, 'subsample': 0.9009856276707946, 'colsample_bytree': 0.8873587278216295, 'learning_rate': 0.01812628862011524, 'n_estimators': 174}. Best is trial 39 with value: 0.3581119118212429.
[I 2023-12-26 16:32:06,384] Trial 78 finished with value: 0.3699288841251913 and parameters: {'booster': 'dart', 'lambda': 2.223721998195954e-05, 'alpha': 0.01733805771126235, 'max_depth': 9, 'subsample': 0.5520278568158997, 'colsample_bytree': 0.9983621629623924, 'learning_rate': 0.06520578532930733, 'n_estimators': 273}. Best is trial 39 with value: 0.3581119118212429.
[I 2023-12-26 16:32:12,329] Trial 79 finished with value: 0.3592552133727836 and parameters: {'booster': 'dart', 'lambda': 1.0969789588438586e-05, 'alpha': 5.39522157983236e-08, 'max_depth': 7, 'subsample': 0.7286688141078844, 'colsample_bytree': 0.

[16:32:44] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "colsample_bytree", "max_depth", "subsample" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




[I 2023-12-26 16:32:44,549] Trial 86 finished with value: 0.4866046320276172 and parameters: {'booster': 'gblinear', 'lambda': 0.002926086150138653, 'alpha': 2.253453504224915e-08, 'max_depth': 6, 'subsample': 0.8571989957515808, 'colsample_bytree': 0.9328362849063314, 'learning_rate': 0.05548917028479323, 'n_estimators': 278}. Best is trial 81 with value: 0.35613359086045393.
[I 2023-12-26 16:32:46,500] Trial 87 finished with value: 0.367554754198414 and parameters: {'booster': 'dart', 'lambda': 8.796758109991034e-07, 'alpha': 1.475626314549499e-07, 'max_depth': 7, 'subsample': 0.7309522187103643, 'colsample_bytree': 0.8436167518682414, 'learning_rate': 0.08881092275910185, 'n_estimators': 260}. Best is trial 81 with value: 0.35613359086045393.
[I 2023-12-26 16:32:53,433] Trial 88 finished with value: 0.3606212627615558 and parameters: {'booster': 'dart', 'lambda': 5.401879706478306e-06, 'alpha': 1.3950981091630666e-06, 'max_depth': 7, 'subsample': 0.7782330163873865, 'colsample_bytre

Best trial:
  Value: 0.3558615060048561
  Params: 
    booster: dart
    lambda: 1.620266765011867e-05
    alpha: 5.635046283951203e-07
    max_depth: 7
    subsample: 0.7103084548037503
    colsample_bytree: 0.9679218656115167
    learning_rate: 0.03883120762643073
    n_estimators: 275


In [64]:
for key, value in main_model.best_trial.params.items():
    print('    {}: {}'.format(key, value))

    booster: dart
    lambda: 1.620266765011867e-05
    alpha: 5.635046283951203e-07
    max_depth: 7
    subsample: 0.7103084548037503
    colsample_bytree: 0.9679218656115167
    learning_rate: 0.03883120762643073
    n_estimators: 275


##### 일조율 제외 Best Parameters: OrderedDict([('colsample_bytree', 0.8828609870729205), ('gamma', 3), ('learning_rate', 0.10496395941543749), ('max_depth', 7), ('n_estimators', 1760), ('subsample', 0.16884742956302834)]) Cross-Validation Score: 0.38765606817069787

##### 일사합 제외 Best Parameters: OrderedDict([('colsample_bytree', 0.5362786341288028), ('gamma', 2), ('learning_rate', 0.3362751905628303), ('max_depth', 3), ('n_estimators', 1819), ('subsample', 0.5797241067611393)]) Cross-Validation Score: 0.3882969067325051

##### 일사합, 일조율, 강수량 제외 Best Parameters: OrderedDict([('colsample_bytree', 0.8725043517301599), ('gamma', 8), ('learning_rate', 0.05309603877595388), ('max_depth', 12), ('n_estimators', 457), ('subsample', 0.4391305289890164)]) Cross-Validation Score: 0.38738144746374126

##### 일조율, 평균풍속 제외 Best Parameters: OrderedDict([('colsample_bytree', 0.6730910375307264), ('gamma', 1), ('learning_rate', 0.056228955612066124), ('max_depth', 10), ('n_estimators', 868), ('subsample', 0.9840445522115008)]) Cross-Validation Score: 0.39200003326815436

##### 일조, 일사, 풍속 제외 Best Parameters: OrderedDict([('colsample_bytree', 0.17537998974601415), ('gamma', 3), ('learning_rate', 0.3143511882114139), ('max_depth', 4), ('n_estimators', 1980), ('subsample', 0.9483620738655641)])Cross-Validation Score: 0.4345285045763367

In [46]:
mds = [최고기온_model, 최저기온_model, 평균습도_model, 평균풍속_model, 일사합_model,
      일조율_model, 강수량_model]

In [50]:
for m, label in zip(mds, labels):
    
    X, y = base_select_label(data, labels, label)
    
    X['months'] = X['months'].astype(int)

    X_train,X_test, y_train, y_test = train_test_data(X,y, 1095)
    
    model = xgb.XGBRegressor(**m.best_trial.params)
    model.fit(X_train, y_train, eval_set=[(X_test, y_test)], early_stopping_rounds=10, verbose=False)
    
    globals()[label+"_md"] = model

[16:47:25] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "colsample_bytree", "max_depth", "subsample" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




In [69]:
X = data[["years", "months", "days","최고기온", "최저기온", "평균습도", "평균풍속","일조율", "강수량"]]
y = data["평균기온"]

X_train,X_test, y_train, y_test = train_test_data(X,y, 365)

In [70]:
model = xgb.XGBRegressor(**main_model.best_trial.params, eval_metric = "mae")
model.fit(X_train, y_train, eval_set=[(X_test, y_test)], early_stopping_rounds=10, verbose=False)

XGBRegressor(alpha=5.635046283951203e-07, base_score=0.5, booster='dart',
             callbacks=None, colsample_bylevel=1, colsample_bynode=1,
             colsample_bytree=0.9679218656115167, early_stopping_rounds=None,
             enable_categorical=False, eval_metric='mae', gamma=0, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', lambda=1.620266765011867e-05,
             learning_rate=0.03883120762643073, max_bin=256,
             max_cat_to_onehot=4, max_delta_step=0, max_depth=7, max_leaves=0,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=275, n_jobs=0, num_parallel_tree=1, predictor='auto',
             random_state=0, ...)

In [71]:
X_test['최고기온'] = 최고기온_model.best_estimator_.predict(X_test[["years", "months", "days"]])
X_test['최저기온'] = 최저기온_model.best_estimator_.predict(X_test[["years", "months", "days"]])
X_test['평균습도'] = 평균습도_model.best_estimator_.predict(X_test[["years", "months", "days"]])
X_test['평균풍속'] = 평균풍속_model.best_estimator_.predict(X_test[["years", "months", "days"]])
# X_test['일사합'] = 일사합_model.best_estimator_.predict(X_test[["years", "months", "days"]])
X_test['일조율'] = 일조율_model.best_estimator_.predict(X_test[["years", "months", "days"]])
X_test['강수량'] = 강수량_model.best_estimator_.predict(X_test[["years", "months", "days"]])

AttributeError: 'Study' object has no attribute 'best_estimator_'

In [72]:
X_test['최고기온'] = 최고기온_md.predict(X_test[["years", "months", "days"]])
X_test['최저기온'] = 최저기온_md.predict(X_test[["years", "months", "days"]])
X_test['평균습도'] = 평균습도_md.predict(X_test[["years", "months", "days"]])
X_test['평균풍속'] = 평균풍속_md.predict(X_test[["years", "months", "days"]])
# X_test['일사합'] = 일사합_md.predict(X_test[["years", "months", "days"]])
X_test['일조율'] = 일조율_md.predict(X_test[["years", "months", "days"]])
X_test['강수량'] = 강수량_md.predict(X_test[["years", "months", "days"]])

In [73]:
y_pred = model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)

print(mae)

## 7개 : 2.6142990210754413
## 일조율 제외 : 2.517694694352885
## 일사합 제외 : 2.504838254349179
## 일사합, 일조율, 강수량 제외 : 2.4861543731534317
## 일조율, 평균풍속 제외 : 2.515234800046437
## 일사합, 일조율, 풍속 제외 : 2.4652675723866238
## 강수량 제외 : 2.5520137255893993
## 일사/일조 제외 : 2.5530409243228296

2.5380074480135146


In [64]:
y_pred = main_model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)

print(mae)

2.4652675723866238


In [59]:
forms = summit_form.copy()
forms.일시 = pd.to_datetime(forms.일시)
forms['days'] = forms.일시.dt.day
forms['months'] = forms.일시.dt.month
forms['years']= forms.일시.dt.year
forms.drop(['일시','평균기온'], axis = 1, inplace = True)

In [86]:
forms['최고기온'] = 최고기온_model.best_estimator_.predict(forms[["years", "months", "days"]])
forms['최저기온'] = 최저기온_model.best_estimator_.predict(forms[["years", "months", "days"]])
forms['평균습도'] = 평균습도_model.best_estimator_.predict(forms[["years", "months", "days"]])
forms['평균풍속'] = 평균풍속_model.best_estimator_.predict(forms[["years", "months", "days"]])
# forms['일사합'] = 일사합_model.best_estimator_.predict(forms[["years", "months", "days"]])
forms['일조율'] = 일조율_model.best_estimator_.predict(forms[["years", "months", "days"]])
forms['강수량'] = 강수량_model.best_estimator_.predict(forms[["years", "months", "days"]])

In [60]:
forms['최고기온'] = 최고기온_md.predict(forms[["years", "months", "days"]])
forms['최저기온'] = 최저기온_md.predict(forms[["years", "months", "days"]])
forms['평균습도'] = 평균습도_md.predict(forms[["years", "months", "days"]])
forms['평균풍속'] = 평균풍속_md.predict(forms[["years", "months", "days"]])
# forms['일사합'] = 일사합_md.predict(forms[["years", "months", "days"]])
forms['일조율'] = 일조율_md.predict(forms[["years", "months", "days"]])
forms['강수량'] = 강수량_md.predict(forms[["years", "months", "days"]])

In [67]:
import joblib
# joblib.dump(최고기온_model, "./models/최고기온_1.joblib")
# joblib.dump(최저기온_model, "./models/최저기온_1.joblib")
# joblib.dump(평균습도_model, "./models/평균습도_1.joblib")
joblib.dump(main_model, "./models/mainmodel(풍속_일조_일사_제외)_1.joblib")

['./models/mainmodel(풍속_일조_일사_제외)_1.joblib']

In [61]:
answer = model.predict(forms)

summit_form["평균기온"] = answer


# mae = mean_absolute_error(y_test, y_pred)

# print(mae)

In [62]:
summit_form.to_csv(f"./answer/xgb_6vars_일사제외_전체_opuna.csv", index = False)

In [4]:
bm = joblib.load("./models/mainmodel(일사합제외)_1.joblib")

In [6]:
bm.best_params_

OrderedDict([('colsample_bytree', 0.5362786341288028),
             ('gamma', 2),
             ('learning_rate', 0.3362751905628303),
             ('max_depth', 3),
             ('n_estimators', 1819),
             ('subsample', 0.5797241067611393)])

In [82]:
# X = data[["years", "months", "days","최고기온", "최저기온", "평균습도", "평균풍속","일사합", "일조율", "강수량"]]
# X = data[["years", "months", "days","최고기온", "최저기온", "평균습도", "평균풍속","일사합","강수량"]]
X = data[["years", "months", "days","최고기온", "최저기온", "평균습도", "평균풍속", "일조율", "강수량"]]
# X = data[["years", "months", "days","최고기온", "최저기온", "평균습도","일사합","강수량"]]
# X = data[["years", "months", "days","최고기온", "최저기온", "평균풍속"]]
# X = data[["years", "months", "days","최고기온", "최저기온", "평균습도", "일조율", "강수량"]]
# X = data[["years", "months", "days","최고기온", "최저기온","평균풍속","일조율", "강수량"]]
y = data["평균기온"]

X_train,X_test, y_train, y_test = train_test_data(X,y, 365)

In [83]:
model = xgb.XGBRegressor(objective="reg:squarederror", **bm.best_params_, n_jobs = -1, random_state=42)
model.fit(X, y)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1,
             colsample_bytree=0.5362786341288028, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, gamma=2, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.3362751905628303,
             max_bin=256, max_cat_to_onehot=4, max_delta_step=0, max_depth=3,
             max_leaves=0, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=1819, n_jobs=-1,
             num_parallel_tree=1, predictor='auto', random_state=42,
             reg_alpha=0, reg_lambda=1, ...)

In [84]:
X_test['최고기온'] = 최고기온_model.best_estimator_.predict(X_test[["years", "months", "days"]])
X_test['최저기온'] = 최저기온_model.best_estimator_.predict(X_test[["years", "months", "days"]])
X_test['평균습도'] = 평균습도_model.best_estimator_.predict(X_test[["years", "months", "days"]])
X_test['평균풍속'] = 평균풍속_model.best_estimator_.predict(X_test[["years", "months", "days"]])
# X_test['일사합'] = 일사합_model.best_estimator_.predict(X_test[["years", "months", "days"]])
X_test['일조율'] = 일조율_model.best_estimator_.predict(X_test[["years", "months", "days"]])
X_test['강수량'] = 강수량_model.best_estimator_.predict(X_test[["years", "months", "days"]])

In [77]:
y_pred = model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)

print(mae)

2.5299242353878197
